# GPU

In [ ]:
import cv2
import torch
from torchvision import transforms, models
from PIL import Image
import numpy as np
from io import BytesIO
import base64
import torch.nn as nn
# from Model import Model

class Model(nn.Module):
    def __init__(self, num_classes=2, latent_dim=2048, lstm_layers=1, hidden_dim=2048, bidirectional=False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained=True)  # Residual Network CNN
        self.feature_extractor = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim, hidden_dim, lstm_layers, bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return self.dp(self.linear1(x))
    
# Load the deepfake detection model
model_path = "saved_best_model/best-checkpoint-model.pt"
model = torch.load(model_path)
# model.load_state_dict(torch.load(model_path))
model.eval()
# model.cuda()

# Transform settings for the model input
im_size = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

def process_frame(frame):
    """Preprocess frame for model input."""
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # Convert frame to PIL format
    frame_tensor = transform(frame_pil).unsqueeze(0).cuda()  # Apply transforms and add batch dimension
    return frame_tensor

def predict(frame_tensor):
    """Run frame through the model and get the prediction."""
    with torch.no_grad():
        logits = model(frame_tensor)
        sm = nn.Softmax(dim=1)
        probabilities = sm(logits)
        confidence, prediction = torch.max(probabilities, dim=1)
    
    return prediction.item(), confidence.item() * 100

# OpenCV video capture
cap = cv2.VideoCapture(0)  # Capture video from the default camera (use 0 for the default camera)

while True:
    ret, frame = cap.read()  # Capture frame-by-frame
    if not ret:
        break

    # Process the frame
    frame_tensor = process_frame(frame)
    
    # Get prediction from the model
    prediction, confidence = predict(frame_tensor)
    
    # Display the prediction on the frame
    label = "Real" if prediction == 1 else "Fake"
    label_text = f"Prediction: {label}, Confidence: {confidence:.2f}%"
    
    # Put text on the frame
    cv2.putText(frame, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if prediction == 1 else (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Deepfake Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture when everything is done
cap.release()
cv2.destroyAllWindows()


# CPU

In [ ]:
import cv2
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
from io import BytesIO
import torch.nn as nn
from Model import Model

class Model(nn.Module):
    def __init__(self, num_classes=2, latent_dim=2048, lstm_layers=1, hidden_dim=2048, bidirectional=False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained=True)  # Residual Network CNN
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim, hidden_dim, lstm_layers, bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.model(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return self.dp(self.linear1(x))

# Load the deepfake detection model
model_path = "/Users/karthiksagar/DeepFake-Detection/saved_best_model/best-checkpoint-model.pt"
# model = Model()

# Ensure the model is loaded on CPU
model=torch.load(model_path, map_location=torch.device('cpu'))
model.eval()  # Set model to evaluation mode

# Transform settings for the model input
im_size = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

def process_frame(frame):
    """Preprocess frame for model input."""
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # Convert frame to PIL format
    frame_tensor = transform(frame_pil).unsqueeze(0)  # Apply transforms and add batch dimension (no .cuda())
    return frame_tensor

def predict(frame_tensor):
    """Run frame through the model and get the prediction."""
    with torch.no_grad():  # No need for gradients during inference
        logits = model(frame_tensor)  # Model forward pass (no .cuda())
        sm = nn.Softmax(dim=1)
        probabilities = sm(logits)
        confidence, prediction = torch.max(probabilities, dim=1)
    
    return prediction.item(), confidence.item() * 100

# OpenCV video capture
cap = cv2.VideoCapture(0)  # Capture video from the default camera (use 0 for the default camera)

while True:
    ret, frame = cap.read()  # Capture frame-by-frame
    if not ret:
        break

    # Process the frame
    frame_tensor = process_frame(frame)
    
    # Get prediction from the model
    prediction, confidence = predict(frame_tensor)
    
    # Display the prediction on the frame
    label = "Real" if prediction == 1 else "Fake"
    label_text = f"Prediction: {label}, Confidence: {confidence:.2f}%"
    
    # Put text on the frame
    cv2.putText(frame, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if prediction == 1 else (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Deepfake Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture when everything is done
cap.release()
cv2.destroyAllWindows()
